In [1]:
from IPython import get_ipython
ipython = get_ipython()
if ipython is not None:
    ipython.magic("%load_ext autoreload")
    ipython.magic("%autoreload 2")

import os
import sys
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('..')
import torch
import re

import acdc
from utils.prune_utils import get_3_caches, split_layers_and_heads
from acdc.TLACDCExperiment import TLACDCExperiment
from acdc.acdc_utils import TorchIndex, EdgeType
import numpy as np
import torch as t
from torch import Tensor
import einops
import itertools

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly
from rich import print as rprint
from rich.table import Table

from jaxtyping import Float, Bool
from typing import Callable, Tuple, Union, Dict, Optional

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

/tmp/ipykernel_14631/1027636265.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%load_ext autoreload")
/tmp/ipykernel_14631/1027636265.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%autoreload 2")


Device: cuda


# Model Setup

In [2]:
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# Dataset Setup

In [3]:
from ioi_dataset import IOIDataset, format_prompt, make_table
N = 25
clean_dataset = IOIDataset(
    prompt_type='mixed',
    N=N,
    tokenizer=model.tokenizer,
    prepend_bos=False,
    seed=1,
    device=device
)
corr_dataset = clean_dataset.gen_flipped_prompts('ABC->XYZ, BAB->XYZ')

make_table(
  colnames = ["IOI prompt", "IOI subj", "IOI indirect obj", "ABC prompt"],
  cols = [
    map(format_prompt, clean_dataset.sentences),
    model.to_string(clean_dataset.s_tokenIDs).split(),
    model.to_string(clean_dataset.io_tokenIDs).split(),
    map(format_prompt, clean_dataset.sentences),
  ],
  title = "Sentences from IOI vs ABC distribution",
)

                                      Sentences from IOI vs ABC distribution                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ IOI prompt                              ┃ IOI subj ┃ IOI indirect obj ┃ ABC prompt                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ When Victoria and Jane got a snack at   │ Jane     │ Victoria         │ When Victoria and Jane got a snack at   │
│ the store, Jane decided to give it to   │          │                  │ the store, Jane decided to give it to   │
│ Victoria                                │          │                  │ Victoria                                │
│                                         │          │                  │                                         │
│ When Sullivan and Rose got a necklace   │ Sullivan │ Rose             │ When Sullivan and Rose got a necklace   │
│ at the garden, Sullivan decided to give │          │                  │ at the garden, Sullivan decided to give │
│ it to Rose                              │          │                  │ it to Rose                              │
│                                         │          │                  │                                         │
│ When Alan and Alex got a drink at the   │ Alex     │ Alan             │ When Alan and Alex got a drink at the   │
│ store, Alex decided to give it to Alan  │          │                  │ store, Alex decided to give it to Alan  │
│                                         │          │                  │                                         │
│ Then, Jessica and Crystal had a long    │ Jessica  │ Crystal          │ Then, Jessica and Crystal had a long    │
│ argument, and afterwards Jessica said   │          │                  │ argument, and afterwards Jessica said   │
│ to Crystal                              │          │                  │ to Crystal                              │
│                                         │          │                  │                                         │
│ Then, Jonathan and Kevin were working   │ Kevin    │ Jonathan         │ Then, Jonathan and Kevin were working   │
│ at the school. Kevin decided to give a  │          │                  │ at the school. Kevin decided to give a  │
│ necklace to Jonathan                    │          │                  │ necklace to Jonathan                    │
│                                         │          │                  │                                         │
└─────────────────────────────────────────┴──────────┴──────────────────┴─────────────────────────────────────────┘

# Metric Setup

In [4]:
def ave_logit_diff(
    logits: Float[Tensor, 'batch seq d_vocab'],
    ioi_dataset: IOIDataset,
    per_prompt: bool = False
):
    '''
        Return average logit difference between correct and incorrect answers
    '''
    # Get logits for indirect objects
    io_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.io_tokenIDs]
    s_logits = logits[range(logits.size(0)), ioi_dataset.word_idx['end'], ioi_dataset.s_tokenIDs]
    # Get logits for subject
    logit_diff = io_logits - s_logits
    return logit_diff if per_prompt else logit_diff.mean()

with t.no_grad():
    clean_logits = model(clean_dataset.toks)
    corrupt_logits = model(corr_dataset.toks)
    clean_logit_diff = ave_logit_diff(clean_logits, clean_dataset).item()
    corrupt_logit_diff = ave_logit_diff(corrupt_logits, corr_dataset).item()

def ioi_metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    ioi_dataset: IOIDataset = clean_dataset
 ):
    patched_logit_diff = ave_logit_diff(logits, ioi_dataset)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

def negative_abs_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs(ioi_metric(logits))
    
# Get clean and corrupt logit differences
with t.no_grad():
    clean_metric = ioi_metric(clean_logits, corrupt_logit_diff, clean_logit_diff, clean_dataset)
    corrupt_metric = ioi_metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, corr_dataset)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

Clean direction: 2.805180072784424, Corrupt direction: 1.326385736465454
Clean metric: 1.0, Corrupt metric: 0.0


# Run Experiment

In [5]:
from ACDCPPExperiment import ACDCPPExperiment
import numpy as np
THRESHOLDS = [-10] #np.logspace(-5, 1, num=20, base=5)
# I'm just using one threshold so I can move fast!

model.reset_hooks()
RUN_NAME = 'acdc_edges'

acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=clean_dataset.toks,
    corr_data=corr_dataset.toks,
    acdc_metric=negative_abs_ioi_metric,
    acdcpp_metric=ioi_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=True,
    attr_absolute_val=True,
    save_graphs_after=0,
    run_acdcpp=False,
    run_acdc=True,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)

pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()

Set up model hooks


  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


dict_keys(['blocks.11.hook_resid_post', 'blocks.11.hook_mlp_out', 'blocks.11.hook_mlp_in', 'blocks.11.attn.hook_result', 'blocks.11.attn.hook_q', 'blocks.11.hook_q_input', 'blocks.11.attn.hook_k', 'blocks.11.hook_k_input', 'blocks.11.attn.hook_v', 'blocks.11.hook_v_input', 'blocks.10.hook_mlp_out', 'blocks.10.hook_mlp_in', 'blocks.10.attn.hook_result', 'blocks.10.attn.hook_q', 'blocks.10.hook_q_input', 'blocks.10.attn.hook_k', 'blocks.10.hook_k_input', 'blocks.10.attn.hook_v', 'blocks.10.hook_v_input', 'blocks.9.hook_mlp_out', 'blocks.9.hook_mlp_in', 'blocks.9.attn.hook_result', 'blocks.9.attn.hook_q', 'blocks.9.hook_q_input', 'blocks.9.attn.hook_k', 'blocks.9.hook_k_input', 'blocks.9.attn.hook_v', 'blocks.9.hook_v_input', 'blocks.8.hook_mlp_out', 'blocks.8.hook_mlp_in', 'blocks.8.attn.hook_result', 'blocks.8.attn.hook_q', 'blocks.8.hook_q_input', 'blocks.8.attn.hook_k', 'blocks.8.hook_k_input', 'blocks.8.attn.hook_v', 'blocks.8.hook_v_input', 'blocks.7.hook_mlp_out', 'blocks.7.hook_ml

No edge 32923
Adding sender hooks...
Running ACDC
No edge 32923
New metric: -0.9999995231628418

Node: cur_parent=TLACDCInterpNode(blocks.11.hook_mlp_out, [:]) (self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:]))

Metric after removing connection to blocks.11.hook_mlp_out [:] is -0.9877183437347412 (and current metric -0.9999995231628418)
Result is 0.012281179428100586...so keeping connection
No edge 32923

Node: cur_parent=TLACDCInterpNode(blocks.11.attn.hook_result, [:, :, 0]) (self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:]))

Metric after removing connection to blocks.11.attn.hook_result [:, :, 0] is -1.0032655000686646 (and current metric -0.9999995231628418)
Result is -0.003265976905822754...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.11.attn.hook_result, [:, :, 1]) (self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:]))

Metric after removing connection to blocks.11.attn.hook_result [:, :, 1] is -0.99372643232345

Metric after removing connection to blocks.10.attn.hook_result [:, :, 10] is -0.745106041431427 (and current metric -0.9999995231628418)
Result is 0.2548934817314148...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.10.attn.hook_result, [:, :, 11]) (self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:]))

Metric after removing connection to blocks.10.attn.hook_result [:, :, 11] is -0.9975106716156006 (and current metric -0.9999995231628418)
Result is 0.002488851547241211...so keeping connection
No edge 32923

Node: cur_parent=TLACDCInterpNode(blocks.9.hook_mlp_out, [:]) (self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:]))

Metric after removing connection to blocks.9.hook_mlp_out [:] is -0.9375362992286682 (and current metric -0.9999995231628418)
Result is 0.062463223934173584...so keeping connection
No edge 32923

Node: cur_parent=TLACDCInterpNode(blocks.9.attn.hook_result, [:, :, 0]) (self.current_node=TLACDCInterpNode(blocks.11.hook_resid_

...

Metric after removing connection to blocks.0.attn.hook_result [:, :, 1] is -0.9964269399642944 (and current metric -0.9999995231628418)
Result is 0.0035725831985473633...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.0.attn.hook_result, [:, :, 2]) (self.current_node=TLACDCInterpNode(blocks.10.hook_mlp_in, [:]))

Metric after removing connection to blocks.0.attn.hook_result [:, :, 2] is -0.9982141256332397 (and current metric -0.9999995231628418)
Result is 0.0017853975296020508...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.0.attn.hook_result, [:, :, 3]) (self.current_node=TLACDCInterpNode(blocks.10.hook_mlp_in, [:]))

Metric after removing connection to blocks.0.attn.hook_result [:, :, 3] is -0.9966724514961243 (and current metric -0.9999995231628418)
Result is 0.0033270716667175293...so keeping connection

Node: cur_parent=TLACDCInterpNode(blocks.0.attn.hook_result, [:, :, 4]) (self.current_node=TLACDCInterpNode(blocks.10.hook_mlp_in, [:]))

Metric 

/opt/conda/lib/python3.10/site-packages/pygraphviz/agraph.py:1407: RuntimeWarning: dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.118976 to fit

  warnings.warn(b"".join(errors).decode(self.encoding), RuntimeWarning)


We moved to  blocks.10.attn.hook_result[:, :, 11]
No edge 32923
No edge 32923
New metric: -0.9999995231628418
No edge 32923
No edge 32923
No edge 32923
We moved to  blocks.10.attn.hook_result[:, :, 10]
No edge 32923
No edge 32923
New metric: -0.9999995231628418
No edge 32923
No edge 32923
No edge 32923
We moved to  blocks.10.attn.hook_result[:, :, 9]
No edge 32923
No edge 32923
New metric: -0.9999995231628418
No edge 32923
No edge 32923
No edge 32923
We moved to  blocks.10.attn.hook_result[:, :, 8]
No edge 32923
No edge 32923
New metric: -0.9999995231628418
No edge 32923
No edge 32923
No edge 32923
We moved to  blocks.10.attn.hook_result[:, :, 7]
No edge 32923
No edge 32923
New metric: -0.9999995231628418
No edge 32923
No edge 32923
No edge 32923
We moved to  blocks.10.attn.hook_result[:, :, 6]
No edge 32923
No edge 32923
New metric: -0.9999995231628418
No edge 32923
No edge 32923
No edge 32923
We moved to  blocks.10.attn.hook_result[:, :, 5]
No edge 32923
No edge 32923
New metric: -0.

Metric after removing connection to blocks.10.hook_k_input [:, :, 9] is -1.0037320852279663 (and current metric -0.9999995231628418)
Result is -0.0037325620651245117...so keeping connection
No edge 32923
We moved to  blocks.10.attn.hook_k[:, :, 8]
No edge 32923
No edge 32923
New metric: -0.9999995231628418

Node: cur_parent=TLACDCInterpNode(blocks.10.hook_k_input, [:, :, 8]) (self.current_node=TLACDCInterpNode(blocks.10.attn.hook_k, [:, :, 8]))

Metric after removing connection to blocks.10.hook_k_input [:, :, 8] is -1.0001307725906372 (and current metric -0.9999995231628418)
Result is -0.00013124942779541016...so keeping connection
No edge 32923
We moved to  blocks.10.attn.hook_k[:, :, 7]
No edge 32923
No edge 32923
New metric: -0.9999995231628418

Node: cur_parent=TLACDCInterpNode(blocks.10.hook_k_input, [:, :, 7]) (self.current_node=TLACDCInterpNode(blocks.10.attn.hook_k, [:, :, 7]))

Metric after removing connection to blocks.10.hook_k_input [:, :, 7] is -1.8799278736114502 (and cu

  0%|          | 0/1 [15:54:03<?, ?it/s]

KeyboardInterrupt



In [ ]:
import json
with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'w') as f:
    json.dump(acdc_pruned_attrs, f)
with open(f'{RUN_NAME}_num_passes.json', 'w') as f:
    json.dump(num_passes, f)

In [8]:
import json
with open(f'{RUN_NAME}_acdcpp_scores.json', 'w') as f:
    json.dump(acdcpp_pruned_attrs, f)

In [7]:
np.quantile(list(acdcpp_pruned_attrs[0.5].values()), [0, 0.25, 0.5, 0.75, 0.9, 1])

array([0.00000000e+00, 8.23954397e-05, 2.85615824e-04, 9.29769565e-04,
       3.00903956e-03, 1.31640387e+00])

In [ ]:
import json

for thresh in pruned_heads.keys():
    pruned_heads[thresh][0] = list(pruned_heads[thresh][0])
    pruned_heads[thresh][1] = list(pruned_heads[thresh][1])
    
cleaned_pp_attrs = {}
for thresh in acdcpp_pruned_attrs.keys():
    cleaned_pp_attrs[thresh] = {}
    for (parent, child), attr in acdcpp_pruned_attrs[thresh].items():
        cleaned_pp_attrs[thresh]\
        [f'{child.hook_point_name}{child.replace_parens(child.index)}{parent.hook_point_name}{parent.replace_parens(parent.index)}'] = attr
        
# Cleaning the edges
for thresh in edges_after_acdcpp.keys():
    edges_after_acdcpp[thresh] = list(edges_after_acdcpp[thresh])
    edges_after_acdc[thresh] = list(edges_after_acdc[thresh])
    
APPEND = False

if APPEND:
    with open(f'{RUN_NAME}_pruned_heads.json', 'r') as f:
        old_pruned_heads = json.load(f)
    with open(f'{RUN_NAME}_acdcpp_pruned_attrs.json', 'r') as f:
        old_acdcpp_pruned_attrs = json.load(f)
    with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'r') as f:
        old_acdc_pruned_attrs = json.load(f)
    with open(f'{RUN_NAME}_num_passes.json', 'r') as f:
        old_num_passes = json.load(f)
    with open(f'{RUN_NAME}_edges_after_acdcpp.json', 'r') as f:
        old_edges_after_acdcpp = json.load(f)
    with open(f'{RUN_NAME}_edges_after_acdc.json', 'r') as f:
        old_edges_after_acdc = json.load(f)
    
    pruned_heads = pruned_heads.update(old_pruned_heads)
    cleaned_pp_attrs = cleaned_pp_attrs.update(old_acdcpp_pruned_attrs)
    acdc_pruned_attrs = acdc_pruned_attrs.update(old_acdc_pruned_attrs)
    num_passes = num_passes.update(old_num_passes)
    edges_after_acdcpp = edges_after_acdcpp.update(old_edges_after_acdcpp)
    edges_after_acdc = edges_after_acdc.update(old_edges_after_acdc)

with open(f'{RUN_NAME}_pruned_heads.json', 'w') as f:
    json.dump(pruned_heads, f)
with open(f'{RUN_NAME}_num_passes.json', 'w') as f:
    json.dump(num_passes, f)
with open(f'{RUN_NAME}_acdcpp_pruned_attrs.json', 'w') as f:
    json.dump(cleaned_pp_attrs, f)
with open(f'{RUN_NAME}_acdc_pruned_attrs.json', 'w') as f:
    json.dump(acdc_pruned_attrs, f)

with open(f'{RUN_NAME}_edges_after_acdcpp.json', 'w') as f:
    json.dump(edges_after_acdcpp, f)
with open(f'{RUN_NAME}_edges_after_acdc.json', 'w') as f:
    json.dump(edges_after_acdc, f)

In [ ]:
# get the 2 fwd and 1 bwd caches; cache "normalized" and "result" of attn layers
clean_cache, corrupted_cache, clean_grad_cache = get_3_caches(
    model, 
    clean_dataset.toks,
    corr_dataset.toks,
    metric=negative_ioi_metric,
    mode = "edge",
)

In [ ]:
clean_head_act = split_layers_and_heads(clean_cache.stack_head_results(), model=model)
corr_head_act = split_layers_and_heads(corrupted_cache.stack_head_results(), model=model)

In [ ]:
stacked_grad_act = torch.zeros(
    3, # QKV
    model.cfg.n_layers,
    model.cfg.n_heads,
    clean_head_act.shape[-3], # Batch
    clean_head_act.shape[-2], # Seq
    clean_head_act.shape[-1], # D
)

for letter_idx, letter in enumerate("qkv"):
    for layer_idx in range(model.cfg.n_layers):
        stacked_grad_act[letter_idx, layer_idx] = einops.rearrange(clean_grad_cache[f"blocks.{layer_idx}.hook_{letter}_input"], "batch seq n_heads d -> n_heads batch seq d")

In [ ]:
results = {}

for upstream_layer_idx in range(model.cfg.n_layers):
    for upstream_head_idx in range(model.cfg.n_heads):
        for downstream_letter_idx, downstream_letter in enumerate("qkv"):
            for downstream_layer_idx in range(upstream_layer_idx+1, model.cfg.n_layers):
                for downstream_head_idx in range(model.cfg.n_heads):
                    results[
                        (
                            upstream_layer_idx,
                            upstream_head_idx,
                            downstream_letter,
                            downstream_layer_idx,
                            downstream_head_idx,
                        )
                    ] = (stacked_grad_act[downstream_letter_idx, downstream_layer_idx, downstream_head_idx].cpu() * (clean_head_act[upstream_layer_idx, upstream_head_idx] - corr_head_act[upstream_layer_idx, upstream_head_idx]).cpu()).sum()

In [ ]:
sorted_results = sorted(results.items(), key=lambda x: x[1].abs(), reverse=True)

In [ ]:
print("Top 10 most important edges:")
for i in range(10):
    print(
        f"{sorted_results[i][0][0]}:{sorted_results[i][0][1]} -> {sorted_results[i][0][3]}:{sorted_results[i][0][4]}",
    )